# 문장 평가
- [잘 설명해준 사이트](https://jrc-park.tistory.com/273)
- 생성된 문장을 평가하는 방식은 크게 2가지 (Reference가 Ground truth로 보면 되는 듯)
  1. Rouge: Reference 문장의 단어가 Generated 문장에 포함되는 정도
     - Text Summarization
  2. **BLEU**: Generated 문장의 단어가 Reference Sentence에 포함되는 정도
     - Machine Translation

## 코드 예시

In [38]:
from nltk.translate.bleu_score import sentence_bleu
reference = [["this", "is", "the", "sample"]]
candidate = ['this', "is", "the", "sample"]
score1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)) # 1.0

score1

1.0

# N-gram
- 위 sentence_bleu 함수의 weights 인자에 들어가는것이 N-gram 인자이며 (1, 0, 0, 0)은 1-gram token들로만 평가를 진행한다는 의미
  - 1-gram
    - this, is, the, sample
  - 2-gram
    - this is, is the, the sample
  - 3-gram
    - this is the, is the sample
  - 4-gram
    - this is the sample
- gram 수가 올라갈수록 **순서를 정확히 고려**해야 하므로 높은 점수를 받기가 쉽지 않아짐

In [48]:
import warnings
warnings.filterwarnings('ignore')

reference = [["나는", "너를", "사랑해", "정말로"]]
candidate = ["나는", "사랑해", "정말로", "너를"]
candidate_v2 = ["나는", "사랑해", "너를", "정말로"]
gram1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
gram2 = sentence_bleu(reference, candidate, weights=(0, 1, 0, 0))
gram_v2 = sentence_bleu(reference, candidate_v2, weights=(0, 1, 0, 0)) # 1.0

print(f'1-gram score: {gram1}, 2-gram score: {gram2}, 2-gram_v2 score: {gram_v2}')

1-gram score: 1.0, 2-gram score: 0.3333333333333333, 2-gram_v2 score: 2.2250738585072626e-308


- 2-gram의 경우 "사랑해", "정말로" 를 통해 2-gram 에 포함되어 있는 문장을 맞췄지만 2-gram_v2의 경우에는 그렇지 못해서 0 값(표기상으론 아주 작은값)을 받은 걸 알 수 있음

# Easy-OCR BLEU 기반 성능 테스트
- 실시간으로 이미지를 계속 읽어와서 테스트를 진행하고자 한다

## 영상 저장 코드

In [1]:
import datetime
import cv2

device_id = 1

capture = cv2.VideoCapture(device_id)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
record = False

while True:
    if(capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        capture.open(device_id)

    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)

    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    key = cv2.waitKey(33)

    if key == 27:       # ESC
        break
    elif key == 26:     # Ctrl - Z
        print("캡쳐")
        cv2.imwrite("D:/" + str(now) + ".png", frame)
    elif key == 24:     # Ctrl - X
        print("녹화 시작")
        record = True
        video = cv2.VideoWriter("D:/" + str(now) + ".avi", fourcc, 20.0, (frame.shape[1], frame.shape[0]))
    elif key == 3:      # Ctrl - C
        print("녹화 중지")
        record = False
        video.release()
        
    if record == True:
        video.write(frame)

capture.release()
cv2.destroyAllWindows()

녹화 시작


## 성능 산출 코드

In [2]:
import easyocr
import cv2
import numpy as np
import warnings
import time
warnings.filterwarnings('ignore')
from nltk.translate.bleu_score import sentence_bleu

def BLEU_mean(source:str, ref:list):
    mean_val = 0
    count = 0
    # cam = cv2.VideoCapture(0)     # if you want to use webcam
    cam = cv2.VideoCapture(source)        # if you want to read video file
    reader = easyocr.Reader(['ko', 'en'], gpu=True)

    reference = [ref]

    while True:
        check,frame = cam.read()
        if not check:
            print("비디오 종료")
            break
        
        count += 1

        cv2.imshow('video', frame)
        result = reader.readtext(frame)
        # candidate = []

        if len(result) != 0:        # 뭔가 잡았다면
            text = []
            for obj in result:
                # bbox = obj[0]
                text.append(obj[1])
                # confidence = obj[2]
            bleu_score = sentence_bleu(reference, text, weights=(1, 0, 0, 0)) # 1.0
            
            if bleu_score != 0.0:       # 많은 예측값들 중 정답 단어를 예측한게 있다면
                mean_val += 1.0
            
            else:
                pass

            print(f"인식한 글자: {text}")
            
        key = cv2.waitKey(1)
        if key == 27:
            break

    cam.release()
    cv2.destroyAllWindows
    return mean_val / count, count

source = 'D:/15_20-33-53.avi'
ref = ['딱풀']
val, frames = BLEU_mean(source, ref)
print(f"Mean score of BLEU: {val}, Total Frame: {frames}")

인식한 글자: ['Ao5', '으', "'따물", '초강터']
인식한 글자: ['MoS', '9', "'따물", '초강다']
인식한 글자: ['MeS', '9', "'따물", '초강다']
인식한 글자: ['Mo5', "'따물", '95', '초강다']
인식한 글자: ['Mo5', "'따물", '95', '초강터']
인식한 글자: ['Mo5', '드5', "'따물", '초강리']
인식한 글자: ['Mo5', '드5', "'따물", '초강터']
인식한 글자: ['A5', '드5', "'따물", '초강리']
인식한 글자: ['A', '9', "'따물", '초강리']
인식한 글자: ['M5', '9', "'따물", '초강리']
인식한 글자: ['Me5', "'따물", '초강적']
인식한 글자: ['Mo5', "'따물", '초강터']
인식한 글자: ['M5', '따물', '9', '초강리']
인식한 글자: ['Ae5', '우', '초강력', '딱풀']
인식한 글자: ['Me5', "'따물", '드', '초강리']
인식한 글자: ['45', '$', "'딱물", '초강되']
인식한 글자: ['M', '우', "'따물", '초강적']
인식한 글자: ['Mo5', '$', "'따물", '초강터']
인식한 글자: ['Mo5', '9', '딱풀', '초강디']
인식한 글자: ['Mo5', '93', "'따물", '초강리']
인식한 글자: ['Mo5', '9', "'", '딱물', '초강리']
인식한 글자: ['Mo5', '95', "'딱물", '초강터']
인식한 글자: ['Mo5', '9', "'따물", '초강터']
인식한 글자: ['4o5', '$', "'딱물", '초강터']
인식한 글자: ['Mo5', '9', 'S3', '따풀', '초강리']
인식한 글자: ['4o5', '도?', '딱물', '9', '초강터']
인식한 글자: ['4o5', 'S', "'따물", '초강리']
인식한 글자: ['MS', '9', '따물', '초강리']
인식한 글자: ['Mos', '따풀', 

## 실험결과 정리

|품목 이름|평균 BLEU 스코어|프레임 수|
|---|---|---|
|Cetaphil|0.295|325|
|만능크리너|0.88|443|
|정관장|0.48|434|
|딱풀|0.18|437|